# About this
This is an implementation of the RSA algorithm made for fun. Based on default Python tools (such as math and random libraries).
___
Includes key generation functions (be careful, you may get a bad key using it, because prime numbers takes randomly and not pass any tests. OpenSSL is better choice);  
message encoding and decoding functions by PKCS#1 protocol;  
encryption and decryption functions.
___
Keys processed as (a, b) tuples!

## import block

In [17]:
import math
import random

## functions for working with bytes

In [18]:
def how_many_bytes(i):
    '''
    i - int.
    Returns the minimum number of bytes needed to represent i
    '''
    return math.ceil(math.log2(i)/8)

In [19]:
def string_to_bytes(string):
    return bytes(string, encoding='utf-8')

In [20]:
def bytes_to_string(b):
    return str(b, encoding='utf-8')

In [21]:
def bytes_to_int(b):
    return int.from_bytes(b, byteorder='big')

In [22]:
def int_to_bytes(i):
    return i.to_bytes(length=how_many_bytes(i), byteorder='big')

In [23]:
def get_ps(length):
    '''Returns a pseudo random byte sequence without null values'''
    return bytes([random.randint(2, 255) for i in range(length)])

## auxiliary mathematical functions

In [24]:
def euclid_ext(a, b):
    '''Extended euclidean algorithm'''
    u, v = (a, 1, 0), (b, 0, 1)
    while v[0] > 0:
        q = u[0] // v[0]
        u, v = v, (u[0]-q*v[0], u[1]-q*v[1], u[2]-q*v[2])
    return u

In [25]:
def invert(a, m):
    '''
    a, m - int.
    Returns the multiplicative inverse of a modulo m
    '''
    u = euclid_ext(a, m)
    assert u[0] == 1
    return u[1] % m

In [26]:
def solovay_strassen_test(n, k=10):
    '''
    n - int > 2; k - number of checks.
    Returns False if n isn't prime and True if n is probable prime
    '''
    if n % 2 == 0:
        return False
    for i in range(k):
        a = random.randint(2, n-1)
        if euclid_ext(a, n)[0] > 1:
            return False
        if pow(a, (n-1)//2, n) != J(a, n) % n:
            return False
    return True

In [27]:
def J(a, b):
    '''
    a, b - int.
    Returns the Jacobi character (a / b)
    '''
    if euclid_ext(a, b)[0] != 1:
        return 0
    r = 1
    if a < 0:
        a = -a
        if b % 4 == 3:
            r = -r
    while a != 0:
        t = 0
        while a % 2 == 0:
            t += 1
            a = a//2
        if t % 2 != 0:
            if b % 8 in (3, 5):
                r = -r
        if a % 4 == 3 and b % 4 == 3:
            r = -r
        a, b = b % a, a
    return r

## keys generation block

In [28]:
def get_prime_numbers(n=2, lower_limit=2**1024, upper_limit=2**2048):
    '''
    Return list of prime numbers of n elements
    from range (lower_limit, upper_limit)
    '''
    ans = []
    for i in range(n):
        while True:
            a = random.randint(lower_limit, upper_limit)
            if solovay_strassen_test(a):
                ans.append(a)
                break
    return ans

In [29]:
def get_public_key(p, q, e_lower_limit=2**15, e_upper_limit=2**20):
    '''
    p, q - prime numbers (int).
    Returns tuple (e, n): e is prime number
    from range (e_lower_limit, e_upper_limit),
    n is composition of p and q
    '''
    n = p * q
    m = (p - 1) * (q - 1)
    while True:
        e = get_prime_numbers(n=1, lower_limit=e_lower_limit, upper_limit=e_upper_limit)[0]
        if euclid_ext(e, m)[0] == 1:
            break
    return (e, n)

In [30]:
def get_secret_key(p, q, e):
    '''
    p, q, e - prime numbers (int).
    Returns tuple (d, n): d is multiplicative inverse of e,
    n is composition of p and q
    '''
    n = p * q
    m = (p - 1) * (q - 1)
    d = invert(e, m)
    return (d, n)

## coding block

In [31]:
def encode(plain_msg, key):
    '''
    Encode ANCII by PKCS#1 protocol.
    plain_msg - str; key - RSA key (typle of e and n).
    Returns EM byte sequence
    '''
    byted_msg = string_to_bytes(plain_msg)
    total_bytes = how_many_bytes(key[1])
    num_of_msg_bytes = how_many_bytes(bytes_to_int(byted_msg))
    num_of_ps_bytes = total_bytes - num_of_msg_bytes - 3
    ps = get_ps(num_of_ps_bytes)
    return bytes([0, 2]) + ps + bytes([0]) + byted_msg

In [32]:
def decode(em):
    '''
    Decode ANCII by PKCS#1 protocol.
    em - EM byte sequence.
    Returns decoded string
    '''
    idx = 0
    for b in em:
        if b == 0 and idx > 0:
            break
        idx += 1
    msg = em[idx+1:]
    return bytes_to_string(msg)

## RSA main function

In [33]:
def crypt(msg, key):
    '''
    Realisation of RSA encrypt/decrypt algorithm.
    msg - int; key - tuple.
    Returns encrypted/decrypted value (int)
    '''
    return pow(msg, key[0], key[1])

## encryption / decryption wrapping functions

In [34]:
def encrypt_msg(msg, public_key):
    '''
    msg - str, puplic_key - typle (e, n) of RSA public key.
    Returns encrypted int value
    '''
    em = encode(msg, public_key)
    m = bytes_to_int(em)
    return crypt(m, public_key)

In [35]:
def decrypt_msg(cipher_msg, secret_key):
    '''
    cipher_msg - int, puplic_key - typle (d, n) of RSA secret key.
    Returns decrypted str value
    '''
    m = crypt(cipher_msg, secret_key)
    em = int_to_bytes(m)
    return decode(em)

# Example

In [37]:
#may takes a few seconds
p, q = get_prime_numbers()

In [47]:
#print(p)
#print(q)
print("length of p: {}\nlength of q: {}".format(len(str(p)), len(str(q))))

length of p: 616
length of q: 616


In [48]:
pub_key = get_public_key(p, q)    #tuple (e, n)
sec_key = get_secret_key(p, q, pub_key[0])    #tuple (d, n)

In [54]:
msg = "Hi, GitHub"
encrypted_msg = encrypt_msg(msg, pub_key)
decrypted_msg = decrypt_msg(encrypted_msg, sec_key)
decrypted_msg == msg

True

In [55]:
encrypted_msg

9589158054788708542848201716238094571746765104182699640399924704499573468715371165800574300807945869227236934030490936407464224253447238306021518130361049035370840627263371672416942672881523445999393032226731576843618093506130848053937998410734729508042576713551816668377369630351257640409917339118495076832274580587372931634521786375887647426710354649092515593435267607833195856741324641208605101033469408585310097788229099709463188224793744914437932141438364033029036070994056938145685090446902002149838107905819515648421024426868188342167418562015842148478657908838281049046942828785130487168725780778267164359700371865980596861752814131901012755380416342263222585709034268367581398815343789091478029696353692798784897566588634484726160657154886012559219307810505204113274617631789665097286574518288559289763279166932945104781870342193124765811151206426376288765418777965023720059067050480608680249724061493338833183450845498725006339795262399923234184070265416258381945208838637369761512444641708

In [56]:
decrypted_msg

'Hi, GitHub'